In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

# Set up Chrome options for headless browsing
chrome_options = Options()
service = Service('/usr/local/bin/chromedriver')  # <-- use the full path here
driver = webdriver.Chrome(service=service, options=chrome_options)

class TestVerrasearchdownload():
    
    def setup_method(self, method):
        self.service = Service('/usr/local/bin/chromedriver')
        self.driver = webdriver.Chrome(service=self.service, options=chrome_options)
        self.vars = {}
  
    def teardown_method(self, method):
        self.driver.quit()
  
    def test_verrasearchdownload(self):
        # 1. Go to the registry
        self.driver.get("https://registry.verra.org/app/search/VCS")
        self.driver.set_window_size(1383, 1415)

        # 2. Click the filter button
        wait = WebDriverWait(self.driver, 10)  # Wait up to 10 seconds
        filter_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".form-group:nth-child(4) .fas"))
        )
        filter_button.click()
        time.sleep(2)

        # 3. Choose the project type from the dropdown
        dropdown = self.driver.find_element(By.ID, "search_sel_proj_type")
        dropdown.find_element(By.XPATH, "//option[contains(text(), 'Energy industries')]").click()

        # 4. Click the "search" button
        self.driver.find_element(By.CSS_SELECTOR, ".btn-primary:nth-child(1)").click()

        # (Optional) Click the "download CSV" button if needed
        # self.driver.find_element(By.CSS_SELECTOR, ".fa-file-csv").click()
        # ✅ Step 3: Wait for Links to Be Present
        try:
            links = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href^='/app/projectDetail/VCS/']"))
            )
            print(f"Found {len(links)} project links.")
        except Exception as e:
            print(f"No links found. Error: {e}")
            self.driver.quit()
            return  # or exit()

        # Extract the hrefs
        hrefs = [link.get_attribute('href') for link in links]
        hrefs = hrefs[:1]
        print(f"Found {len(hrefs)} project links")

        # Loop through each project link, open, extract paragraphs, return
        extracted_data = []
        for href in hrefs:
            try:
                self.driver.get(href)
                time.sleep(2)

                # Wait for the project detail text
                div_element = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-text.p-3"))
                )
                paragraph_texts = [p.text for p in div_element.find_elements(By.TAG_NAME, "p")]
                extracted_data.append({"url": href, "text": paragraph_texts})
                print(f"Extracted {len(paragraph_texts)} paragraphs from {href}")

                # Go back to the results page
                self.driver.back()
                time.sleep(2)

            except Exception as e:
                print(f"Error processing {href}: {e}")

        # Convert to a DataFrame
        df = pd.DataFrame(extracted_data)
        return df


if __name__ == "__main__":
    # Instantiate the test class
    test_instance = TestVerrasearchdownload()
    
    # Manually run setup_method
    test_instance.setup_method(None)
    
    # Call the test
    result_list = test_instance.test_verrasearchdownload()
    
    # Print out the contents
    print(result_list)
    
    # Teardown/quit the driver
    test_instance.teardown_method(None)


Found 50 project links.
Found 50 project links
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5408
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5410
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5411
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5373
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5340
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5337
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5313
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5312
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5277
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5279
Extracted 1 paragraphs from https://registry.verra.org/app/projectDetail/VCS/5274
Extracted 1 paragraphs from https://registry.verra.

In [ ]:
# Cleaning and transformation

result_list["id"] = result_list.apply(lambda x: int(x[0].split("/")[-1]), axis=1)
# convert result_list['text'] to a string
result_list["text"] = result_list["text"].apply(lambda x: " ".join(x))
result_list.head()

/var/folders/j2/yjk_0cz112g3l8vv2_013tmr0000gn/T/ipykernel_6398/4167730192.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result_list["id"] = result_list.apply(lambda x: int(x[0].split("/")[-1]), axis=1)


In [22]:
result_list.to_csv("result_list.csv", index=False)